In [70]:
!pip install transformers
!pip install sentencepiece

In [71]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers

In [72]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 10

# Labels in our dataset.
labels = ["NOT ENTAILMENT - Unknown", "Entailment", "NOT ENTAILMENT - Contradiction"]

In [73]:
PATH = '/content/drive/MyDrive/Colab Notebooks/ambiguous_knights_knaves.json'

In [74]:
import json
def read_jsonl_file(PATH):
  df = pd.read_json(PATH)

  with open(PATH,'r') as f:
    data = json.loads(f.read())

  df_nested_list = pd.json_normalize(data=data['puzzles'], record_path='QA', meta=['puzzle_text'])
  return df_nested_list

In [75]:
df = pd.DataFrame(read_jsonl_file(PATH))

In [76]:
# SE IMPARTE DATASET-UL IN TRAIN, VALIDARE SI TEST
train_df = df[['qid','puzzle_text', 'question', 'answer']][:700]
valid_df  = df[['qid','puzzle_text', 'question', 'answer']]
test_df  = df[['qid','puzzle_text', 'question', 'answer']]

# Shape of the data
print(f"Total train samples : {train_df.shape[0]}")
print(f"Total validation samples: {valid_df.shape[0]}")
print(f"Total test samples: {test_df.shape[0]}")

Total train samples : 700
Total validation samples: 940
Total test samples: 940


In [77]:
print("Train Target Distribution")
print(train_df.answer.value_counts())

Train Target Distribution
NOT ENTAILMENT - Unknown          536
Entailment                         82
NOT ENTAILMENT - Contradiction     82
Name: answer, dtype: int64


In [78]:
print("Validation Target Distribution")
print(valid_df.answer.value_counts())

Validation Target Distribution
NOT ENTAILMENT - Unknown          730
Entailment                        105
NOT ENTAILMENT - Contradiction    105
Name: answer, dtype: int64


In [79]:
train_df = (
    train_df[train_df.answer != "-"]
    .sample(frac=1.0, random_state=42)
    .reset_index(drop=True)
)
valid_df = (
    valid_df[valid_df.answer != "-"]
    .sample(frac=1.0, random_state=42)
    .reset_index(drop=True)
)

In [80]:
train_df["label"] = train_df["answer"].apply(
    lambda x: 0 if x == "NOT ENTAILMENT - Contradiction" else 1 if x == "Entailment" else 2
)
y_train = tf.keras.utils.to_categorical(train_df.label, num_classes=3)

valid_df["label"] = valid_df["answer"].apply(
    lambda x: 0 if x == "NOT ENTAILMENT - Contradiction" else 1 if x == "Entailment" else 2
)
y_val = tf.keras.utils.to_categorical(valid_df.label, num_classes=3)

test_df["label"] = test_df["answer"].apply(
    lambda x: 0 if x == "NOT ENTAILMENT - Contradiction" else 1 if x == "Entailment" else 2
)
y_test = tf.keras.utils.to_categorical(test_df.label, num_classes=3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [81]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [82]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False 

    bert_output = bert_model(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state
    pooled_output = bert_output.pooler_output
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    max_pooling = tf.keras.layers.MaxPooling1D(2)(bi_lstm)
    dense1 = tf.keras.layers.Dense(1, activation='relu')(max_pooling)  
    conv1 = tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(dense1)   
    dense2 = tf.keras.layers.Dense(10, activation='sigmoid')(conv1) 
    conv2 = tf.keras.layers.Conv1D(filters=64, kernel_size=5, padding='same', activation='relu')(dense2)
    conv3 = tf.keras.layers.Conv1D(filters=32, kernel_size=5, padding='same', activation='relu')(conv2)
    conv4 = tf.keras.layers.Conv1D(filters=16, kernel_size=4, padding='same', activation='relu')(conv3)
    dense3 = tf.keras.layers.Dense(20, activation='sigmoid')(conv4) 
    conv5 = tf.keras.layers.Conv1D(filters=8, kernel_size=3, padding='same', activation='relu')(conv4)
    conv6 = tf.keras.layers.Conv1D(filters=4, kernel_size=3, padding='same', activation='relu')(conv5)
    dense4 = tf.keras.layers.Dense(20, activation='sigmoid')(conv6) 
    flatten = tf.keras.layers.Flatten()(dense4)
    output = tf.keras.layers.Dense(3, activation="softmax")(flatten)
    model2 = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model2.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="binary_crossentropy",       # am schimbat loss function
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model2.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Strategy: <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7f80bae24490>
Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model_10 (TFBertModel)  TFBaseModelOutputWi  109482240  ['input_ids[0][0]',      

In [83]:
train_data = BertSemanticDataGenerator(
    train_df[["puzzle_text", "question"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    valid_df[["puzzle_text", "question"]].values.astype("str"),
    y_val,
    batch_size=batch_size,
    shuffle=False,
)

In [84]:
history = model2.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/10
21/21 [==============================] - ETA: 0s - loss: 0.4558 - acc: 0.7634

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


21/21 [==============================] - 43s 974ms/step - loss: 0.4558 - acc: 0.7634 - val_loss: 0.4193 - val_acc: 0.7748
Epoch 2/10
21/21 [==============================] - 16s 759ms/step - loss: 0.4324 - acc: 0.7634 - val_loss: 0.4170 - val_acc: 0.7748
Epoch 3/10
21/21 [==============================] - 17s 847ms/step - loss: 0.4333 - acc: 0.7664 - val_loss: 0.4254 - val_acc: 0.7748
Epoch 4/10
21/21 [==============================] - 16s 751ms/step - loss: 0.4299 - acc: 0.7664 - val_loss: 0.4239 - val_acc: 0.7748
Epoch 5/10
21/21 [==============================] - 15s 750ms/step - loss: 0.4305 - acc: 0.7634 - val_loss: 0.4193 - val_acc: 0.7748
Epoch 6/10
21/21 [==============================] - 15s 754ms/step - loss: 0.4383 - acc: 0.7634 - val_loss: 0.4293 - val_acc: 0.7748
Epoch 7/10
21/21 [==============================] - 16s 756ms/step - loss: 0.4300 - acc: 0.7664 - val_loss: 0.4135 - val_acc: 0.7748
Epoch 8/10
21/21 [==============================] - 16s 756ms/step - loss: 0.426

In [85]:
# Unfreeze the bert_model.
bert_model.trainable = True
# Recompile the model to make the change effective.
model2.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model2.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model_10 (TFBertModel)  TFBaseModelOutputWi  109482240  ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_masks[0][0]',  

In [86]:
history = model2.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/10
21/21 [==============================] - 51s 2s/step - loss: 0.4297 - accuracy: 0.7693 - val_loss: 0.4223 - val_accuracy: 0.7748
Epoch 2/10
21/21 [==============================] - 25s 1s/step - loss: 0.4280 - accuracy: 0.7664 - val_loss: 0.4192 - val_accuracy: 0.7748
Epoch 3/10
21/21 [==============================] - 25s 1s/step - loss: 0.4312 - accuracy: 0.7604 - val_loss: 0.4171 - val_accuracy: 0.7748
Epoch 4/10
21/21 [==============================] - 25s 1s/step - loss: 0.4259 - accuracy: 0.7649 - val_loss: 0.4154 - val_accuracy: 0.7748
Epoch 5/10
21/21 [==============================] - 25s 1s/step - loss: 0.4204 - accuracy: 0.7693 - val_loss: 0.4146 - val_accuracy: 0.7748
Epoch 6/10
21/21 [==============================] - 25s 1s/step - loss: 0.4227 - accuracy: 0.7664 - val_loss: 0.4139 - val_accuracy: 0.7748
Epoch 7/10
21/21 [==============================] - 25s 1s/step - loss: 0.4222 - accuracy: 0.7664 - val_loss: 0.4135 - val_accuracy: 0.7748
Epoch 8/10
21/21 [==

In [87]:
test_data = BertSemanticDataGenerator(
    test_df[["puzzle_text", "question"]].values.astype("str"),
    y_test,
    batch_size=batch_size,
    shuffle=False,
)
model2.evaluate(test_data, verbose=1)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


29/29 [==============================] - 9s 294ms/step - loss: 0.4139 - accuracy: 0.7737


[0.41388189792633057, 0.7737069129943848]

In [88]:
def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = model2.predict(test_data[0])[0]
    idx = np.argmax(proba)
    proba = f"{proba[idx]: .2f}%"
    pred = labels[idx]
    return pred, proba

In [89]:
sentence1 = "On the island where each inhabitant is either a knave or a knight , knights always tell the truth while knaves always lie . You meet three inhabitants : Alice , Rex and Bob .  Rex claims that it is false that Bob is a knave . Bob says that he is a knight or Alice is a knight . Can you determine who is a knight and who is a knave ?"
sentence2 = "Is Rex the knight ?"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


('NOT ENTAILMENT - Contradiction', ' 0.92%')